# Introdução

Nosso projeto consiste em avaliar o processo de visto americano para trabalho, o H1B, e se dependendo de certos fatores uma pessa receberia o visto ou não. Para tirar este visto é necessário passar informações como, o nome do lugar de trabalho, com o que a pessoa trabalhará, se é um trabalho de tempo integral, o salário, e a cidade onde será o trabalho (traduzido para latitude e longitude), todos dados contidos no nosso dataset. Porém havia um problema nosso dataset, obtido no Kaggle havia 3 milhões de linhas, um arquivo muito grande, fazendo com que fique difícil e demorado para se trabalhar. Por esse motivo cortamos o dataset, no qual o processo está em um outro notebook chamado Shuffle dataset.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
#Abrindo o dataframe
df = pd.read_csv('Dataframe.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,0,1805963,1,"SAGARSOFT, INC",Computer and Information Systems Managers,PROJECT MANAGER,Y,69222.4,2013.0,"SHELTON, CONNECTICUT",-73.093164,41.316486
1,1,767976,0,MOUNT AUBURN HOSPITAL,PHYSICAL THERAPISTS,PHYSICAL THERAPIST,Y,89648.0,2015.0,"CAMBRIDGE, MASSACHUSETTS",-71.109733,42.373616
2,2,376800,1,"ERP ANALYSTS, INC.","SOFTWARE DEVELOPERS, APPLICATIONS",SOFTWARE PROGRAMMER/ANALYST,Y,78957.0,2016.0,"CREVE COEUR, MISSOURI",-90.422618,38.660885
3,3,1233483,1,"ORCHESTRA TECHNOLOGY, INC",NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,IMS CORE QA TEST ENGINEER,Y,63190.0,2015.0,"BELLEVUE, WASHINGTON",-122.201516,47.610150
4,4,2235324,0,"SAP INDUSTRIES, INC.",Computer Systems Analysts,APPLICATIONS CONSULTANT IV,Y,97635.0,2012.0,"IRVING, TEXAS",-96.948894,32.814018


Na célula abaixo, o tratamento dos dados é feito. Em primeiro lugar as colunas Unnamed: 0 e Unnamed: 0.1, são retiradas, essas correspondem ao index atual e ao index antes do embaralhamento feito no outro notebook. O local de trabalho, WORKSITE, também é retirado uma vez que esse já está no dataset em forma de Latitude e Longitude, variáves quantitavas. Essas então são transformadas para valores de 0 a 1 para facilitar a forma de se trabalhar, bem como a variável de trabalho em tempo intergral, FULL_TIME_POSITION, também é transformada, de N ou Y, para 0 ou 1. JOB_TITLE é retirado pois a variável SOC_NAME, do Inglês Standard Occupational Classification (SOC), já corresponde ao trabalho ocupado porém em uma forma de classificação padrão e essa então é transormada para uma váriavel quantitativa através do metodo de One-hot Encoding. Por fim o ano é retirado uma vez que queremos prever o visto de anos futuros então não se pode levar em conta os anos passados.  

In [3]:
def dummify(data, column_name):
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

def mapp (value, start1, stop1, start2, stop2):
    return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))


df = df.drop(["WORKSITE","YEAR","JOB_TITLE","Unnamed: 0","Unnamed: 0.1"], axis=1)
df["SOC_NAME"] = df["SOC_NAME"].apply(lambda x: x.upper())

df = df.drop(["EMPLOYER_NAME"], axis=1)

df["FULL_TIME_POSITION"] = df["FULL_TIME_POSITION"].apply(lambda x: 0 if x=="N" else 1)



latmin = df["lat"].min()
latmax = df["lat"].max()
lonmin = df["lon"].min()
lonmax = df["lon"].max()

df["lat"] = df["lat"].apply(lambda x: mapp(x, latmin, latmax, 0, 1))
df["lon"] = df["lon"].apply(lambda x: mapp(x, lonmin, lonmax, 0, 1))

df = dummify(df, "SOC_NAME")

In [4]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_ABLE SEAMEN,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTORS,SOC_NAME_ACTUARIES,...,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, SECONDARY SCHOOL",SOC_NAME_WATCH REPAIRERS,SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WEIGHERS, MEASURERS, CHECKERS, AND SAMPLERS,","SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS","SOC_NAME_WOODWORKERS, ALL OTHER",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,69222.4,0.279211,0.542393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,89648.0,0.285744,0.562959,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,78957.0,0.222129,0.490728,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,63190.0,0.117451,0.664836,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,97635.0,0.200632,0.376977,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O X e y do modelo, baseando-se no CASE_STATUS(se foi aprovado ou não) são separados em base para treinamentos e base para teste, sendo a divisão feita de 67 e 33 por cento. Em seguida é aplicado o modelo, neste caso de regressão logística, uma vez que esse método consegue fazer a predição de para valores a partir das nossas variáves que foram transformadas em quantitativas.

In [5]:
X = df.drop(["CASE_STATUS"], axis=1).values
y = df["CASE_STATUS"].values

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=RANDOM_SEED)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((67000, 811), (67000,), (33000, 811), (33000,))

Na celula abaixo, faz-se a implementação do classificador RandomForest. Este modelo foi escolhido por nós pois, ao se analisar a chamada "Flow Chart" do Sklearn, a categoria que mais se alinhava com nosso modelo era a "ensemble classifier". O classificador RandomForest faz parte desta categoria. 

Após implementarmos o modelo, calculamos sua precisão. Para fazermos isso, separamos o dataframe em duas amostras de 50000: a primeira para treinar o classificador e a segunda para testá-lo de fato. Depois disso, comparamos os resultados obtidos com os reais, sendo que nosso classificador 'acertava' em 65% dos casos. 

Na celula abaxio, vê-se o 'flow chart' do Sklearn. Foi a partir dele que escolhemos qual seria o melhor modelo para nosso problema.


Abaixo, o 'caminho' que percorremos no flow chart até chegarmos no RandomForest

![title](img/FlowChartNosso.png)

In [6]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Precisão: {}%".format(100*accuracy_score(y_test, y_pred)))

Precisão: 65.33939393939394%


In [7]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_ABLE SEAMEN,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTORS,SOC_NAME_ACTUARIES,...,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, SECONDARY SCHOOL",SOC_NAME_WATCH REPAIRERS,SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WEIGHERS, MEASURERS, CHECKERS, AND SAMPLERS,","SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS","SOC_NAME_WOODWORKERS, ALL OTHER",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,69222.4,0.279211,0.542393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,89648.0,0.285744,0.562959,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,78957.0,0.222129,0.490728,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,63190.0,0.117451,0.664836,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,97635.0,0.200632,0.376977,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Nas duas celulas abaixo, com intuito de melhorar a análise da eficiência do modelo, adicionamos a matriz de confusão que compara os valores reais com os previstos pelo modelo. A primeira tabela mostra os valores do treinamento, enquantos a segunda mostra os de teste. Como podemos ver na primeira, 30418 vezes o previsto foi 0 e o real também, e assim por diante. 

Previsto,0,1,All
Real,,,
0,30418,3016,33434
1,2512,31054,33566
All,32930,34070,67000


In [18]:
pd.crosstab(y_test, model.predict(X_test),rownames=['Real'],colnames =['Previsto'],margins=True)

Previsto,0,1,All
Real,,,
0,11163,5403,16566
1,6035,10399,16434
All,17198,15802,33000


Nas células abaixo esta contido um programa que pergunta ao usuário suas informações e, a partir delas, reponde se ela teria ou não seu visto concedido. Vale ressaltar que o classificador acerta em 65% dos casos, ou seja, a resposta dada por ele ainda tem 35% de chance de estar errada. 

In [11]:
import ipywidgets as widgets

trabalhos = df.columns.values
trabalhos = trabalhos[5:]
temp = []
for i in trabalhos:
    temp.append(i[9:])
    
def perguntas():
    try:
        print("Olá, responda essas perguntas para saber se uma pessoa teria o visto americano de trbalho H1B.")
        f = int(input("A pessoa tem trabalho integral?(0-Não/1-Sim) "))
        w = float(input("Qual o salário da pessoa? USD$/Ano "))
        lo = float(input("Qual a longitude da cidade(apenas do EUA)? "))
        la = float(input("Qual a latitude da cidade(apenas do EUA)? "))
    except:
        print("Erro em algum valor passado. ")
        
    return f, w, lo, la

In [20]:
#responde, com base no modelo se uma pessoa com as informações fornecidas seria aprovada ou não.


fulltime, wage, longi, lati = perguntas()

longi = mapp(longi, lonmin, lonmax, 0, 1)
lati = mapp(lati, latmin, latmax, 0, 1)
w = widgets.Dropdown(
    options=temp,
    description="Trabalho:",
    disabled=False
)
b = widgets.Button(
    description="OK",
    disabled=False
)
display(w, b)
j = ""
def click(a):
    global j
    b.layout.display = "none"
    w.disabled = True
    j = w.value
    
    job = "SOC_NAME_" + j
    
    d = {"FULL_TIME_POSITION":fulltime, "PREVAILING_WAGE":wage, "lon":longi, "lat":lati, job:1}
    l=[]

    for i in df.columns:
        if i != "CASE_STATUS":
            if i in d:
                l.append(d[i])
            else:
                l.append(0)

    # print(l)
    resultado = model.predict([l])
    # print(resultado)

    if resultado == [1]:
        print("Essa pessoa teria sido aprovada")
    else:
        print("Essa pessoa teria sido reprovada")

b.on_click(click)


Olá, responda essas perguntas para saber se uma pessoa teria o visto americano de trbalho H1B.
A pessoa tem trabalho integral?(0-Não/1-Sim) 1
Qual o salário da pessoa? USD$/Ano 40000
Qual a longitude da cidade(apenas do EUA)? -70
Qual a latitude da cidade(apenas do EUA)? 40


Dropdown(description='Trabalho:', options=('ABLE SEAMEN', 'ACCOUNTANTS', 'ACCOUNTANTS AND AUDITORS', 'ACTORS', 'ACTUARIES', 'ADMINISTRATIVE SERVICES MANAGERS', 'ADULT BASIC AND SECONDARY EDUCATION AND LITERACY', 'ADULT BASIC AND SECONDARY EDUCATION AND LITERACY T', 'ADULT LITERACY, REMEDIAL EDUCATION, AND GED TEACHE', 'ADVERTISING AND PROMOTIONS MANAGERS', 'ADVERTISING SALES AGENTS', 'AEROSPACE ENGINEERING AND OPERATIONS TECHNICIANS', 'AEROSPACE ENGINEERS', 'AGENTS AND BUSINESS MANAGERS OF ARTISTS,', 'AGENTS AND BUSINESS MANAGERS OF ARTISTS, PERFORMER', 'AGRICULTURAL AND FOOD SCIENCE TECHNICIANS', 'AGRICULTURAL AND FOOD SCIENTISTS', 'AGRICULTURAL CROP FARM MANAGERS', 'AGRICULTURAL ENGINEERS', 'AGRICULTURAL INSPECTORS', 'AGRICULTURAL SCIENCES TEACHERS, POSTSECONDARY', 'AGRICULTURAL TECHNICIANS', 'AGRICULTURAL WORKERS, ALL OTHER', 'AIRCRAFT MECHANICS AND SERVICE TECHNICIANS', 'AIRLINE PILOTS, COPILOTS, AND FLIGHT ENGINEERS', 'ALL OTHER FOOD PROCESSING WORKERS', 'ANESTHESIOLOGISTS', 'ANIMAL BREEDERS', 'ANIMAL SCIENTISTS', 'ANIMAL TRAINERS', 'ANTHROPOLOGISTS AND ARCHEOLOGISTS', 'ANTHROPOLOGY AND ARCHEOLOGY TEACHERS,', 'ANTHROPOLOGY AND ARCHEOLOGY TEACHERS, POSTSECONDAR', 'APPRAISERS AND ASSESSORS OF REAL ESTATE', 'APPRAISERS, REAL ESTATE', 'ARBITRATORS, MEDIATORS, AND CONCILIATORS', 'ARCHITECTS, EXCEPT LANDSCAPE AND NAVAL', 'ARCHITECTURAL AND CIVIL DRAFTERS', 'ARCHITECTURAL AND ENGINEERING MANAGERS', 'ARCHITECTURAL DRAFTERS', 'ARCHITECTURE TEACHERS, POSTSECONDARY', 'ARCHIVISTS', 'AREA, ETHNIC, AND CULTURAL STUDIES TEACHERS,', 'AREA, ETHNIC, AND CULTURAL STUDIES TEACHERS, POSTS', 'ART DIRECTORS', 'ART, DRAMA, AND MUSIC TEACHERS, POSTSECONDARY', 'ARTISTS AND RELATED WORKERS, ALL OTHER', 'ASTRONOMERS', 'ATHLETES AND SPORTS COMPETITORS', 'ATHLETIC TRAINERS', 'ATMOSPHERIC AND SPACE SCIENTISTS', 'ATMOSPHERIC, EARTH, MARINE, AND SPACE SCIENCES', 'ATMOSPHERIC, EARTH, MARINE, AND SPACE SCIENCES TEA', 'ATTORNEY', 'AUDIO AND VIDEO EQUIPMENT TECHNICIANS', 'AUDIO-VISUAL AND MULTIMEDIA COLLECTIONS', 'AUDIO-VISUAL AND MULTIMEDIA COLLECTIONS SPECIALIST', 'AUDIO-VISUAL COLLECTIONS SPECIALISTS', 'AUDIOLOGISTS', 'AUDITORS', 'AUTOMOTIVE BODY AND RELATED REPAIRERS', 'AUTOMOTIVE MASTER MECHANICS', 'AUTOMOTIVE SERVICE TECHNICIANS AND MECHANICS', 'AUTOMOTIVE SPECIALTY TECHNICIANS', 'AVIONICS TECHNICIANS', 'BAKERS', 'BAKERS, BREAD AND PASTRY', 'BARTENDERS', 'BILLING AND POSTING CLERKS', 'BILLING, COST, AND RATE CLERKS', 'BIOCHEMISTS', 'BIOCHEMISTS & BIOPHYSICISTS', 'BIOCHEMISTS AND BIOPHYSICISTS', 'BIOLOGICAL SCIENCE TEACHERS, POSTSECONDARY', 'BIOLOGICAL SCIENTIST, ALL OTHERS', 'BIOLOGICAL SCIENTISTS', 'BIOLOGICAL SCIENTISTS, ALL OTHER', 'BIOLOGICAL SCIENTISTS, ALL OTHERS', 'BIOLOGICAL TECHNICIAN', 'BIOLOGICAL TECHNICIANS', 'BIOLOGISTS', 'BIOMEDICAL ENGINEERS', 'BIOPHYSICISTS', 'BOOKKEEPING, ACCOUNTING, AND AUDITING CLERKS', 'BROADCAST NEWS ANALYSTS', 'BROADCAST TECHNICIANS', 'BROKERAGE CLERKS', 'BUDGET ANALYSTS', 'BUSINESS INTELLIGENCE ANALYSTS', 'BUSINESS OPERATIONS SPECIALISTS, ALL OTHER', 'BUSINESS OPERATIONS SPECIALISTS, ALL OTHER*', 'BUSINESS TEACHERS, POSTSECONDARY', 'BUYERS AND PURCHASING AGENTS, FARM PRODUCTS', 'CABINETMAKERS AND BENCH CARPENTERS', 'CALIBRATION AND INSTRUMENTATION TECHNICIANS', 'CAMERA OPERATORS, TELEVISION, VIDEO, AND MOTION', 'CAMERA OPERATORS, TELEVISION, VIDEO, AND MOTION PI', 'CAPTAINS, MATES, AND PILOTS OF WATER VESSELS', 'CARDIOVASCULAR TECHNOLOGISTS AND TECHNICIANS', 'CAREER/TECHNICAL  EDUCATION TEACHERS, MIDDLE', 'CAREER/TECHNICAL EDUCATION TEACHERS, MIDDLE SCHOOL', 'CAREER/TECHNICAL EDUCATION TEACHERS, SECONDARY', 'CAREER/TECHNICAL EDUCATION TEACHERS, SECONDARY SCH', 'CARGO AND FREIGHT AGENTS', 'CARPENTERS', 'CARTOGRAPHERS AND PHOTOGRAMMETRISTS', 'CHEFS AND HEAD COOKS', 'CHEMICAL ENGINEERS', 'CHEMICAL TECHNICIANS', 'CHEMISTRY TEACHERS, POSTSECONDARY', 'CHEMISTS', 'CHIEF EXECUTIVES', 'CHILD, FAMILY, AND SCHOOL SOCIAL WORKERS', 'CHILDCARE WORKERS', 'CHIROPRACTORS', 'CHOREOGRAPHERS', 'CIVIL DRAFTERS', 'CIVIL ENGINEERING TE

Button(description='OK', style=ButtonStyle())

Essa pessoa teria sido aprovada


## Conclusão

A partir dos resultados obtidos, percebe-se que o modelo tem uma precisão de quase 65%. Desta forma, conclui-se que o modelo não é tão preciso. Portanto, se uma pessoa deseja saber se terá seu visto para os EUA concedido, basta que ela use o modelo com suas informações. A resposta obtida terá 65% de probabilidade de condizer com a verdadeira.

Um problema grande que encontramos no projeto foi encontrar um modelo que fosse bom para nosso problema. Isso porquê a maioria dos modelos davam uma precisão abaixo de 60%. Desta forma, o modelo RandomForest foi o melhor que encontramos, visto que sua precisão foi de 65%.


## Futuras Iterações

Para uma possível futura iteração, consideraríamos testar o modelo com algum outro modelo, que reteornaria uma precisão maior. Alem disso, algo que torna o 'qusetionário' um pouco 'chato' é que muitas vezes o usuário não sabe a longitude e latitude de onde mora. Assim, seria interessante, futuramente adicionar algum recurso que descobre a latitude e longitude de uma cidade a partir de seu nome, que seria digitado pelo usuário. 

## Separação do Trabalho

Shuffle do dataset- Freddy 

Importação e limpeza do dataset- Michel

Separação treinamento/teste e aplicação do Modelo- Freddy/Michel

Matriz de confusão- Michel

Análise exploratória inicial- Freddy

Análise exploratória- Michel

Teste para um usuário- Freddy

